In [2]:
%reload_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True
%matplotlib inline
  
import numpy as np
import matplotlib.pyplot as plt


In [3]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [72]:
# the following cells will use the rai framework for robot control, simulation and motion planning
# https://github.com/MarcToussaint/rai
import sys
import os 
print(os.getcwd())
# add the folder where libry.so is located to the path. Otherwise the import will crash.
sys.path.append('../../ry/')
sys.path.append('../../')
sys.path.append('../')
import libry as ry
import time
import gc
import os
from collections import defaultdict
import utils

# add simulation. Note: if the string argument is not an empty string, a ROS node is started
# and the joint state topics of the real baxter are subscribed. This won't work if you can't connect to Baxter.
# In order to connect to Baxter, uncomment the next 2 lines and set the correct IP address:
os.environ["ROS_MASTER_URI"] = "http://thecount.local:11311/"
os.environ["ROS_IP"] = "129.69.216.204"

/home/niklas/git/uni/robotics-course/practical/clean


In [73]:
# clear views, config and operate by setting shared pointers to 0. Otherwise the notebook has to be restarted,
# which is pretty annoying.
C = 0
v = 0
B = 0
gc.collect()
    
# initialize config
C = ry.Config()
v = C.view()
C.clear()
C.addFile('../../rai-robotModels/baxter/baxter_new.g')
nodeName = ""#"studentMarcsNode"

q_home = C.getJointState()
q_zero = q_home * 0.
B = C.operate(nodeName)
B.sync(C)
C.makeObjectsConvex()
B.sendToReal(False)

In [ ]:
cam = ry.Camera(nodeName, "")

In [108]:
def check_target(targetFrame):
    if not targetFrame in C.getFrameNames():
        frame = C.addObject(name=targetFrame, parent="base_footprint" ,shape=ry.ST.sphere, size=[.01], pos=[0,0,0], color=[0.,0.,1.])
    return C.frame(targetFrame)

def plan_path(targetPos, angle,targetFrame, gripperFrame, steps, time):
    target = check_target(targetFrame)
    rotM = utils.rotz(angle)
    B.sync(C)
    quat = utils.rotm2quat(rotM)
    target.setPosition(targetPos)
    target.setQuaternion(quat)
    pp = C.komo_path(1, 30, 10, False)
    pp.setConfigurations(C)
    pp.clearObjectives()
    pp.addObjective(type=ry.OT.eq, feature=ry.FS.scalarProductYZ, frames=[gripperFrame, targetFrame], target=[0], time=[])
    pp.addObjective(type= ry.OT.eq, feature= ry.FS.scalarProductZZ, frames= [gripperFrame, targetFrame], target= [1], time= [])
    pp.addObjective(type= ry.OT.sos,feature= ry.FS.qItself, frames= [], target= q_home, time= [1.])
    pp.addObjective(type= ry.OT.eq, feature= ry.FS.distance, frames= [gripperFrame, targetFrame], target= [-0.2], time= [.8])
    pp.addObjective(type= ry.OT.eq, feature= ry.FS.positionDiff, frames= [gripperFrame, targetFrame], time= [1.])
    pp.addObjective(type= ry.OT.sos, feature= ry.FS.qItself, frames= [], order= 1,  time= [0, 1])
    pp.optimize(False)
    t = pp.getT()
    path = []
    for i in range(t):
        frames = pp.getConfiguration(i)
        C.setFrameState(frames)
        q = C.getJointState()
        path += [q]
    return path, pp

In [109]:
B.moveHard(q_home)

In [111]:
steps = 30; time = 5
p, _ = plan_path([0.4, 0, 1], 1.2, "ball", "baxterR", steps, time)

In [112]:
B.move(p, [time/steps * i for i in range(len(p))], False)

In [93]:
p

[array([ 0.     ,  0.08001, -0.08001, -1.00001, -1.00001,  1.17001,
        -1.17001,  1.94001,  1.94001, -0.67001,  0.67001,  1.02001,
         1.02001,  0.50001, -0.50001,  0.     ,  0.     ]),
 array([ 0.     ,  0.08001, -0.08001, -1.00001, -1.00001,  1.17001,
        -1.17001,  1.94001,  1.94001, -0.67001,  0.67001,  1.02001,
         1.02001,  0.50001, -0.50001,  0.     ,  0.     ]),
 array([ 0.     ,  0.08001, -0.08001, -1.00001, -1.00001,  1.17001,
        -1.17001,  1.94001,  1.94001, -0.67001,  0.67001,  1.02001,
         1.02001,  0.50001, -0.50001,  0.     ,  0.     ]),
 array([ 0.     ,  0.08001, -0.08001, -1.00001, -1.00001,  1.17001,
        -1.17001,  1.94001,  1.94001, -0.67001,  0.67001,  1.02001,
         1.02001,  0.50001, -0.50001,  0.     ,  0.     ]),
 array([ 0.     ,  0.08001, -0.08001, -1.00001, -1.00001,  1.17001,
        -1.17001,  1.94001,  1.94001, -0.67001,  0.67001,  1.02001,
         1.02001,  0.50001, -0.50001,  0.     ,  0.     ]),
 array([ 0.     ,  0

In [29]:
from webserver import sampleClient

In [ ]:
grasp = sampleClient.predictGQCNN()